# Import

In [ ]:
#!pip install openai

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pprint
import logging

import hopenai
import snippets
import helpers.hdbg as hdbg

hdbg.init_logger()

hdbg.set_logger_verbosity(logging.INFO)

In [7]:
import os; os.environ["OPENAI_API_KEY"] = ""

In [ ]:
if False:
    # Force reloading a module.
    import hopenai
    from importlib import reload
    reload(hopenai)

# Chat

In [8]:
hopenai.get_completion("hello")

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-9p31rOvTTzSSk5i5OoCqNs2oiNQEv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1721953399, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0f03d4f0ee', usage=CompletionUsage(completion_tokens=9, prompt_tokens=12, total_tokens=21))

# Eval prompt

In [125]:
function_tag = "code_snippets2"
transform_tag = "remove_docstring"
prompt_tag = "docstring"
in_outs = snippets.eval_prompt(function_tag, transform_tag, prompt_tag) 

print(snippets.in_outs_to_str(in_outs))

INFO  Processing 3 examples


  0%|                                                                                                                                                             | 0/3 [00:00<?, ?it/s]

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 33%|█████████████████████████████████████████████████▋                                                                                                   | 1/3 [00:03<00:07,  3.79s/it]

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


 67%|███████████████████████████████████████████████████████████████████████████████████████████████████▎                                                 | 2/3 [00:08<00:04,  4.06s/it]

INFO  HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.92s/it]

INFO  Saving results ...


### in.txt ###
def listdir(
    dir_name: str,
    pattern: str,
    only_files: bool,
    use_relative_paths: bool,
    *,
    exclude_git_dirs: bool = True,
    maxdepth: Optional[int] = None,
) -> List[str]:
    hdbg.dassert_dir_exists(dir_name)
    cmd = [f"find {dir_name}", f'-name "{pattern}"']
    if maxdepth is not None:
        cmd.append(f'-maxdepth "{maxdepth}"')
    if only_files:
        cmd.append("-type f")
    if exclude_git_dirs:
        cmd.append(r'-not -path "*/\.git/*"')
    cmd = " ".join(cmd)
    _, output = hsystem.system_to_string(cmd)
    paths = [path for path in output.split("\n") if path != ""]
    _LOG.debug("Found %s paths in %s", len(paths), dir_name)
    _LOG.debug("\n".join(paths))
    if use_relative_paths:
        paths = [os.path.relpath(path, start=dir_name) for path in paths]
    return path

def keep_python_files(
    file_names: List[str], exclude_paired_jupytext: bool
) -> List[str]:
    hdbg.dassert_isinstance(file_n

In [127]:
snippets.in_out_to_files(in_outs)

AttributeError: module 'snippets' has no attribute 'in_out_to_files'

# Assistant

In [ ]:
system = """You are a proficient Python coder and write English very well. 
Given the Python code passed below, improve or add comments to the code.
Each comment should be in imperative form, a full English phrase, and end with a period.
Comments must be for every logical chunk of 4 or 5 lines of Python code.
Do not comment every single line of code and especially logging statements.
"""

# There should be no empty line in the code.

user1 = snippets.get_code_snippet2()

response = hopenai.get_completion(user, system=system)

print(hopenai.response_to_txt(response))

## Query using library

In [ ]:
assistant_name = "coder_assistant"
instructions = "You are an expert Python coder. Use you knowledge base to answer questions about how to write code."

vector_store_name = "Coding style"
file_paths = ["all.coding_style.how_to_guide.md"]

assistant = hopenai.get_coding_style_assistant(
    assistant_name,
    instructions,
    vector_store_name,
    file_paths)

In [ ]:
hopenai.pprint(assistant)

In [ ]:
#question = "What is DRY?"
question = "Should one pay the technical debt?"
messages = hopenai.get_query_assistant(assistant, question)